# **LTI&trade; Advantage** bootcamp notebook

## Introduction

The notebook shows how to interact with the LTI Advantage ecosystem from a tool implementer viewpoint. It interacts with an actual test server which has been built as a platform simulator to support this notebook.


## Setup

Here we just import the libraries that will be needed in this notebook, define some utility functions and constants.

In [79]:
import requests
import json
import jwt
import base64
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from IPython.display import display, Markdown

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# for concise code, return claim prefixed by ims
def fc(claim):
    return "http://imsglobal.org/lti/{0}".format(claim)

def md(mdt):
    display(Markdown(mdt))

# This notebook queries an actual test server. It needs its location:
platform_url='http://localhost:5000'


First we need to get a new tool deployment from the server for this notebook instance to use.
Each tool must have a client_id and a private key that will be used to interact with the platform services and send messages back to the platform.
It also needs the keyset URL that exposes the platform public keys needed to validate the incoming messages.

While this information is required for each tool, how it is obtained by the tool is NOT currently part of the LTI specifications.

In [81]:
r = requests.get(platform_url + '/newtool')
tool_info = json.loads(r.text)

md('### Tool information')
md('Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.')
md('```json\n'+ json.dumps(tool_info, indent=4)+'```')

### Tool information

Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.

```json
{
    "client_id": "10",
    "deployment_id": "deployment_10",
    "key": {
        "e": "AQAB",
        "kty": "RSA",
        "use": "sig",
        "alg": "RS256",
        "d": "HlnyJFrLiVPoqudQnY3MVVIR5kVghabtTog2TU4ugRynM-PcYhC9dQUVc-RC01lD2gitwVW7YAHt1nzRr3kPaqrmwa9K8MBtaRKYqm1SMujLXQJDXkMec-dTG7RgQNvyzfBzB_0FX8M4vZZyiswhBbWnR27MIoF6jrU7z48uFcgnkI9YklDytNGKXjMsSdU8KOa_K8fG9UsYaJXyLPwYmVOBiOeVfppU07QOICNHrfQ9ds0C_94uyFIuXdRLggDaJK957k-ytlAXMWKqiuogh5Doce-xztEdlwAImEj_4MJP_j1SADr5dYvDHCiBtcdKcatwyljuMcOe0sI3cwIyMQ=="
    }
}```

### Getting the public keys from the platform

In order to validate the various messages we will receive from the platform, we need to get the key sets. We'll also transform the keys to the `PEM` format that is used by the `jwt` module to decode the messages.

The test platform exposes its keys in a keyset format at a well-know location (.well-known/jwks.json). Other platform might just communicate the keyset url as part of the tool deployment information.

In [82]:
keyset = json.loads(requests.get(platform_url + '/.well-known/jwks.json').text)

md('#### Platform keyset')
md('```json\n'+ json.dumps(keyset, indent=4)+'```')

platform_keys = {}

# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    


#### Platform keyset

```json
{
    "keys": [
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "wb68Yov1n2pe3wrlBsgUGfQJFOe_7oaya-Y45q_yRjMx2YhO5ZER8mhM81OjRnXMdq7tUJxy-PyHmVJFgTGwZAyokNvDmkPA_1ifVv_c0LnJW9C9BTl2ehljUciv1abwOITKEPcQoQj3ISmGCKMEB-eiQMs5mruunb1AalLnJu_TAfl-BnEhBRznd9thyQcoPUrDD2nfsnmVCoMQJaQzl6EBqCI1d-hK4abNHog_KTMpoe3LrSnUiIjtPWkvWNFv0j1h07HlqhOcWQ6fGHxwLEb1phXhb2xlQ79sIvrJp58cERyMrp6VDb8ozVCwBcgfBE8Eqww8gBCyrMgZtYeflQ==",
            "kid": "1517151491_0"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "0dqZ5VuLqhjOuW7_YFTUn_dogYbwZhJJdafHyvCdwIdGaMrn6E9dkgx_4jSTonCDhn2LZOWVKn-6Es8pW8gI2S3vjIqxe3CryP-fkCR-Y5ZRG9hUjktIE_suoURTk2VbAmYAyXpV-iECn8Rr0KkFfo6Te4H_VU21VPjXBEVANIxTaXtze3dI-6hhtjYnB2KPcgj5PvDAbQh6IytHcVWHxjACYbVXyb0uQvpRwsE5PsDa2P9ZxxFmr6npyuwM9RblyIF7CDSU6ZrNu-ZPBucdkTy78DCedUPmo6dqZa5t1GeNawZJtLIlj_y696C_sARuFsNGIzBvfleWIsoFMYO10w==",
            "kid": "1517151491_1"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "q0lkmWCrMLrB7icb1xvYf1vzKZxmn9Ez5imo2pXPRry7KJdbq3EVrgJ1QClV1mo41Lh6alKf4RK8WitI61IgOI1zDLeNtrj0ef-idXiKS5d3M4VVWuQ80w1fXHeB65ebXpEYH9AZlNDjo9jHahfA_8blYCviK8_v68CTpHWnGSNhGHEUBpy__c_YttsiadVjg7Qf7h9ddmZMXifWPgLtrCWOXbuqHOJ-OKvPEpEvM7lBOaSqqSUxuhQTnAPrXsxJCABPNqpoUBVkLNdfxPnTHDWF-X_kf_gsVRPHuOhUgL-nI3lyoA5l8V4--b-NoWWgI_UzCtGaElI2k6MNEZSdqw==",
            "kid": "1517151491_2"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "29AVqbzp1wXw3s4TFod3cPUo7DNWcWh0NnnSC_Z8ZoZSbz3-9iTP1w_hgwqknALCbtRe0Xz4j9htjEjyA8XdzlPq-B2Ik3mGINT6Ya5AyZaLO7dQ4fEBYUP6v3TuTXhW9z-RYxDoRj3L7mvGvJpU0uqCPJhiZ-pat7truZ82LqAuDkjnJLorQxYnJtlrflBkUKSMixW9-OfByPMMJNnZ0gRHOwMne0m6pyrG_5LNw4EFVwS9X5EQ7tedTJ7ndFVN7GuN5Us7uUC-zWZheoOwXDPqVANksmPGy0U5wV9huS51_qUdzOdGnGrnWgtWSgeS2MtisrFoNCi2ObbNfuCYPw==",
            "kid": "1517151491_3"
        }
    ]
}```

## Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

[Deep Linking](https://www.imsglobal.org/specs/lticiv1p0) is a **UI flow**, it is an important piece that is sometimes missed on 1st reading. The user is redirected from the platform to the tool to pick or create one or multiple piece of content (often, LTI links), and the the tool redirects the UI back to the tool with the actual selection (or an empty payload if nothing was picked or created).

So there are 2 messages:

1. `ContentItemRequestSelection` from the platform to the tool to start the picking/create session. This is a typical platform launch that contains the context and the user information, and what kind of content items may be created in this flow (for example, this flow might indicate it only wants LTI links).
1. `ContentItemSelection` from the tool back to the plaform using the `content_item_return_url` provided in the request.

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request.

![deep linking](assets/lti_advantage_deeplinking.png "LTI Deep Link")

### Setup: Getting a Deep Linking Request

This notebook is not a tool actually launched by the platform, so the test platform as way to give us the token that it would include in an actual HTTP POSTrequest, so we can build a mock POST request including the parameter `post_data`.

In [74]:
r = requests.get("{}/tool/{}/cisr".format(platform_url, tool_info['client_id']))

post_data = {
    'id_token':r.text
}

### Task 1: Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.


In [83]:
# Let's get the kid so we can get the proper public key
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

md('#### Decoded ContentItemSelectionRequest message')
md('```json\n'+ json.dumps(content_item_message, indent=4)+'```')


ExpiredSignatureError: Signature has expired

### Task 2: extract the information to decide which experience to show

If this is the first launch for the user or the course, as a tool you may prompt the user for setup information, including account linking or course setup. Ultimately the user will see the authoring or picking interface that will allow her to create or select the content items to be added to the course.

Some key attributes of the `ContentItemSelectionRequest` will drive the experience:

In [84]:
md('1. What is the current course id? {0}'.format(content_item[c('')]))
md('1. What is the current user id? {0}'.format(content_item['sub']))
md('1. Is this user an instructor? {0}'.format(content_item['sub']))
md('1. What kind of content item can be created? {0}'.format(content_item[c('')]))
md('1. Can I return more than one items to be added? {0}'.format(content_item[c('')]))
md('1. Can I return more than one items to be added? {0}'.format(content_item[c('')]))
md('1. Where should I redirect the browser too when done? {0}'.format(content_item[c('')]))
md('1. Is there any data I must pass back to platform when I return? {0}'.format(content_item[c('')]))

NameError: name 'content_item' is not defined

### Task 3: redirect the user back to the platform with the content item selection

After the end of the interaction, so user is sent back to the platform throught a browser redirection using an HTTP POST containing the JWT `ContentItemResponse` message. In this case, we will return 2 LTI links, one being graded.